# 3. Model Testing (Inference Simulation)

This notebook simulates the inference process on the target hardware (4 CPU, 4GB RAM).
We assume the model has been converted to GGUF format (e.g., `qwen2.5-3b-reminder-bot-q4_k_m.gguf`).

If the GGUF file is not ready, this notebook will fail. 
You can use `llama-cpp-python` to run GGUF models in Python.

## Prerequisites
```bash
pip install llama-cpp-python
```


In [5]:
from llama_cpp import Llama
import json

# Path to your GGUF model
# You must have run the conversion in step 2 or downloaded a GGUF version
MODEL_PATH = "qwen2.5-3b-reminder-bot-q4_k_m.gguf" 

try:
    # Initialize model with 4GB RAM constraint logic
    # n_ctx=2048 context window
    llm = Llama(
        model_path=MODEL_PATH,
        n_ctx=2048,
        n_threads=4,      # 4 CPU cores
        n_gpu_layers=0    # 0 GPU layers (CPU only simulation)
    )
    print("Model loaded successfully on CPU.")
except Exception as e:
    print(f"Error loading model (Did you convert it to GGUF?): {e}")
    llm = None

llama_model_loader: loaded meta data with 28 key-value pairs and 434 tensors from qwen2.5-3b-reminder-bot-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_k i32              = 20
llama_model_loader: - kv   3:                     general.sampling.top_p f32              = 0.800000
llama_model_loader: - kv   4:                      general.sampling.temp f32              = 0.700000
llama_model_loader: - kv   5:                               general.name str              = Qwen2.5 3b Reminder Bot_Merged
llama_model_loader: - kv   6:                           general.finetune str              = reminder-bot_merged
llama_model_loader: - kv   7:

Model loaded successfully on CPU.


In [6]:
import pandas as pd
import random

def extract_reminder(text, context_date="2026-02-18"):
    if not llm:
        return "Model not loaded."
    
    system_prompt = """Ты — система для извлечения параметров напоминаний.
Твоя задача: извлечь текст, дату, время и периодичность из сообщения пользователя и вернуть JSON.
Используй текущую дату (Context Date) для разрешения относительных дат.
Формат: {"text": "...", "date": "YYYY-MM-DD", "time": "HH:MM", "repeat": "..."}"""
    
    user_prompt = f"Context Date: {context_date}\nMessage: \"{text}\"\n\nJSON:"
    
    prompt = f"<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"
    
    output = llm(
        prompt,
        max_tokens=256,
        stop=["<|im_end|>"],
        temperature=0.1,
        echo=False
    )
    
    return output['choices'][0]['text']

# Load random 15 messages from CSV
try:
    df = pd.read_csv("user_messages.csv")
    # Filter for non-empty messages
    df = df.dropna(subset=['message_text'])
    df = df[df['message_text'].str.len() > 10]
    
    # Sample 15 random messages
    sample_df = df.sample(25)
    test_messages = sample_df['message_text'].tolist()
    print(f"Loaded {len(test_messages)} random messages from CSV.")
    
except Exception as e:
    print(f"Error loading CSV, using default examples: {e}")
    test_messages = [
        "Напомни купить молоко завтра в 10 утра",
        "Встреча с боссом 25 февраля в 14:00",
        "Каждый день в 9:00 пить таблетки"
    ]

for msg in test_messages:
    if llm:
        print(f"Input: {msg}")
        # Clean up newlines for cleaner output display
        print(f"Output: {extract_reminder(msg)}")
        print("-" * 20)

Loaded 25 random messages from CSV.
Input: КАЖДОЕ 28 ЧИСЛО ПРОВЕРКА ПРЕМИУМА ТИНЬКОФФ


llama_perf_context_print:        load time =     517.78 ms
llama_perf_context_print: prompt eval time =     516.51 ms /   164 tokens (    3.15 ms per token,   317.52 tokens per second)
llama_perf_context_print:        eval time =    2121.81 ms /    53 runs   (   40.03 ms per token,    24.98 tokens per second)
llama_perf_context_print:       total time =    2683.81 ms /   217 tokens
llama_perf_context_print:    graphs reused =         51
Llama.generate: 127 prefix-match hit, remaining 29 prompt tokens to eval


Output: {"text": "ПРОВЕРКА ПРЕМИУМА ТИНЬКОФФ", "date": "2026-02-28", "time": "00:00", "repeat": "monthly"}
--------------------
Input: 16 ФЕВРАЛЯ ИТОГИ РОЗЫГРЫША


llama_perf_context_print:        load time =     517.78 ms
llama_perf_context_print: prompt eval time =     121.68 ms /    29 tokens (    4.20 ms per token,   238.33 tokens per second)
llama_perf_context_print:        eval time =    1853.27 ms /    46 runs   (   40.29 ms per token,    24.82 tokens per second)
llama_perf_context_print:       total time =    2007.37 ms /    75 tokens
llama_perf_context_print:    graphs reused =         43
Llama.generate: 127 prefix-match hit, remaining 28 prompt tokens to eval


Output: {"text": "Итоги розыгрыша", "date": "2026-02-16", "time": "00:00", "repeat": "none"}
--------------------
Input: КАЖДОЕ 1 ЧИСЛО ОПЛАТА ПОЧТЫ


...

Output: {"text": "итоги розыгрыша", "date": "2026-01-28", "time": null, "repeat": null}
--------------------
Input: [Релизы] 
https://wiki.domrf.ru/pages/viewpAge.action?paGeId=267595334

Банк:
‼️18.01.2026 в 10:00 (воскр)
https://jira.domrf.ru/browse/INTEGRATIO-4771
https://jira.dOmrf.ru/bRowse/INTEGRATIO-4879
еЛк-Эскроу СИнхРониЗациЯ данных из На + доработка проверки бенЕфИцИАров
https://jira.domrf.ru/browse/RFC-9906 
@exkot @hopheylalalei сделайте стРаницу по эТому релизу (по возМожности, лиБо рпосто отпишите есть ли Какие дополнителЬные зАдаЧи пО Конфигугированию в этом релизе) и пропИшите ветки плз в табличке https://wiki.dOmrf.ru/pagEs/viewpAge.Action?pageId=315611369 

 [elK-facade] [athena-adapter] [athena-facade]

@alexzOdiac..


llama_perf_context_print:        load time =     517.78 ms
llama_perf_context_print: prompt eval time =     922.67 ms /   301 tokens (    3.07 ms per token,   326.23 tokens per second)
llama_perf_context_print:        eval time =    2523.44 ms /    62 runs   (   40.70 ms per token,    24.57 tokens per second)
llama_perf_context_print:       total time =    3488.19 ms /   363 tokens
llama_perf_context_print:    graphs reused =         59
Llama.generate: 126 prefix-match hit, remaining 23 prompt tokens to eval


Output: {"text": "Банк: сделайте страницу по этому релизу и пропишите ветки в табличке", "date": "2026-01-18", "time": "10:00", "repeat": "none"}
--------------------
Input: 28  января итоги розЫгрыша


llama_perf_context_print:        load time =     517.78 ms
llama_perf_context_print: prompt eval time =     102.91 ms /    23 tokens (    4.47 ms per token,   223.50 tokens per second)
llama_perf_context_print:        eval time =    1799.13 ms /    45 runs   (   39.98 ms per token,    25.01 tokens per second)
llama_perf_context_print:       total time =    1932.34 ms /    68 tokens
llama_perf_context_print:    graphs reused =         42


Output: {"text": "итоги розЫгрыша", "date": "2026-01-28", "time": "00:00", "repeat": "none"}
--------------------


## Note setup for production

For the actual 4GB/4CPU deployment, you simply need:
1. The `.gguf` file.
2. A lightweight python script using `llama-cpp-python` (like above).
3. No heavy dependencies like `torch` or `transformers`. Just `llama-cpp-python`.
